In [1]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.tools import tool
from langchain_ollama import OllamaLLM
from langchain_chroma import Chroma
from langchain_community.tools import DuckDuckGoSearchResults, YouTubeSearchTool, AzureAiServicesDocumentIntelligenceTool, WikipediaQueryRun

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict
import os
import datetime

/home/sam/Github/365DaysOfData/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
llm = OllamaLLM(model= 'llama3.2', temperature=0.7)

llm.invoke("Hello, Ollama!")

"Hello! It's nice to meet you. I'm here to help with any questions or topics you'd like to discuss. How can I assist you today?"

# 1. contextual q&a with memory
build a chatbot that not only answers questions about a dataset (like pdfs or notes) but also remembers past queries using langgraph’s state nodes.
bonus: add a “forget” command that clears memory.

In [3]:
from langchain_community.document_loaders import PyPDFLoader

doc = PyPDFLoader("/home/sam/Github/365DaysOfData/14-Agentic-AI/code/extras/geeta.pdf")
docs = doc.load()
parser = StrOutputParser()



In [4]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="mxbai-embed-large")
vectorstore = Chroma.from_documents(docs, embedding=embeddings)

### Using Langchain

In [5]:
from langchain_core.prompts import ChatPromptTemplate


chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Bhagavad Gita expert. answer the question based on the context provided. If the context does not help, say 'I don't know'"),
    ("human", "query: {query} context: {context}"),
])

chain = (
    RunnablePassthrough() 
    | (lambda query: {"query": query, "context": "\n".join([doc.page_content for doc in vectorstore.similarity_search(query, k=3)])})
    | chat_prompt
    | llm
    | parser
)

chain.invoke("What is lust, how lust destroy humans??")


# there's no memeory implemented yet

'Based on the provided context from the Bhagavad Gita, lust is described as the greatest enemy of living entities and the primary cause of their entanglement in the material world.\n\nLust is said to be the manifestation of the mode of passion (raja-guña) within an individual. When a living entity comes into contact with the material creation, their eternal love for Kåñëa (the Supreme Personality of Godhead) is transformed into lust, driven by their senses (indriyāëi), mind (manaù), and intelligence (buddhiù).\n\nLust is characterized as an all-devouring sinful enemy that can never be satisfied by material sense gratification. It is said to "cover" or "overwhelm" the embodied soul (dehinam), preventing them from attaining true knowledge (jïänam) and spiritual realization.\n\nThe Bhagavad Gita teaches that when lust is not transformed into love for the Supreme, it can lead to a cycle of craving, dissatisfaction, and ultimately, wrath. However, when an individual transforms their lustful

### Using Langgraph

In [6]:
def retrieve_context(state):
    query = state["messages"][-1].content if state["messages"] else ""
    docs = vectorstore.similarity_search(query, k=3)
    context = "\n".join([doc.page_content for doc in docs])
    return {"context": context}

def response_generation(state):
    context = state.get("context", "")
    query = state["messages"][-1].content if state["messages"] else ""

    # Get the LLM's response
    response = llm.invoke(chat_prompt.format_messages(query=query, context=context, chat_history=state["messages"][:-1]))
    
    # Check the type of response and handle accordingly
    if hasattr(response, 'content'):
        response_text = response.content
    else:
        response_text = str(response)
        
    # Create and return AIMessage with the response text
    return {"messages": state["messages"] + [AIMessage(content=response_text)]}

# Handle commands like 'forget'
def handle_commands(state):
    last_message = state["messages"][-1].content.lower()
    
    if last_message.startswith("/forget"):
        # Clear the history except for this command
        return {"messages": [state["messages"][-1]], "command_executed": True}
    return {"command_executed": False}

In [7]:
# Complete implementation example
class State(TypedDict):
    messages: list
    context: str
    command_executed: bool

graph = StateGraph(State)

# Add all nodes with consistent names
graph.add_node("retrieve", retrieve_context)
graph.add_node("command_handler", handle_commands)
graph.add_node("respond", response_generation)

# Connect START to first node
graph.add_edge(START, "command_handler")

graph.add_conditional_edges(
    "command_handler",
    lambda state: state["command_executed"],
    {False: "retrieve", True: END}
)

graph.add_edge("retrieve", "respond")
graph.add_edge("respond", END)

workflow = graph.compile(checkpointer=MemorySaver())

In [8]:
state = {"messages": [], "context": "", "command_executed": False}

print("Chat with Bhagavad Gita Expert (type '/forget' to clear history, '/exit' to quit)")

while True:
    user_input = input("\nYou: ")
    
    if user_input.lower() == "/exit":
        break
        
    state["messages"] = add_messages(state["messages"], [HumanMessage(content=user_input)])
    
    state = workflow.invoke(
        state,
        config={
            "configurable": {
                "thread_id": "user_conversation_1",  
                "checkpoint_ns": "bhagavad_gita_chat" 
            }
        }
    )
    
    print(f"\nHuman: {state['messages'][-2].content}")
    print(f"AI: {state['messages'][-1].content}")

Chat with Bhagavad Gita Expert (type '/forget' to clear history, '/exit' to quit)

Human: what do you think aobut how this world is going to destroy?
AI: Based on the provided context from the Bhagavad Gita, I believe that this world will be destroyed by those who are driven by demoniac tendencies and engage in activities that are detrimental to the well-being of others. These individuals are described as having "no intelligence" and being devoid of all sense, and they prioritize their own sense gratification over the happiness and prosperity of others.

In particular, the verse mentions the invention of nuclear weapons, which is a modern example of such destructive technology. The Bhagavad Gita suggests that such creations are not meant for peace and prosperity but rather for destruction.

The text also highlights the importance of education in developing the mode of goodness, which can lead to sobriety, full knowledge, and happiness. It warns against the dangers of animal killing and

# 2. multi-agent pattern

Create different specialized agent and extract output from different layers.

In [21]:
from typing import TypedDict
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate

END = "END"

# --- debate state ---
class DebateState(TypedDict):
    messages: list          # full transcript
    next: str               # which agent speaks next
    round: int              # current round
    max_rounds: int         # stop after N rounds
    topic: str              # the debate topic
    sides: dict             # which side each agent defends
    scores: dict            # track scores per agent
    highlights: dict        # best one-liner or argument from each

# --- workflow engine ---
class StateGraph:
    def __init__(self):
        self.nodes = {}
        self.entry_point = None
    
    def add_node(self, name, func):
        self.nodes[name] = func
    
    def set_entry_point(self, name):
        self.entry_point = name
    
    def get_node(self, name):
        return self.nodes.get(name)

    Agent that argues based on science, logic, and evidence

In [ ]:

def science_agent(state: DebateState):
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"You are a science debater. Defend the side: {state['sides']['science']}. "
                   "Respond with one short, sharp paragraph using scientific reasoning and facts. "
                   "Be persuasive but concise."),
        ("user", "{messages}")
    ])
    chain = prompt | llm
    response = chain.invoke({"messages": state["messages"]})
    
    # log message
    state["messages"].append(f"Science: {response}")
    
    # update next turn
    state["next"] = "religion"
    return state


    Agent that argues from religious/God perspective

In [ ]:

def religion_agent(state: DebateState):
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"You are a religious debater. Defend the side: {state['sides']['religion']}. "
                   "Respond with one short, sharp paragraph using religious, moral, or spiritual reasoning. "
                   "Be persuasive but concise."),
        ("user", "{messages}")
    ])
    chain = prompt | llm
    response = chain.invoke({"messages": state["messages"]})
    
    # log message
    state["messages"].append(f"Religion: {response}")
    
    # update next turn
    state["next"] = "evaluator"
    return state


    Harsh evaluator that scores both sides and picks best highlights.

In [ ]:

def evaluator_agent(state: DebateState):    
    # grab last two debater messages
    last_science = [m for m in state["messages"] if m.startswith("Science:")][-1]
    last_religion = [m for m in state["messages"] if m.startswith("Religion:")][-1]
    
    # dummy scoring: could replace with LLM judgment, but here random for demo
    science_score = random.randint(5, 10)
    religion_score = random.randint(5, 10)
    
    state["scores"]["science"] += science_score
    state["scores"]["religion"] += religion_score
    
    # pick "highlight" (just choose whichever scored higher this round)
    if science_score >= religion_score:
        state["highlights"]["science"] = last_science
        highlight = f"Best this round: Science side → {last_science}"
    else:
        state["highlights"]["religion"] = last_religion
        highlight = f"Best this round: Religion side → {last_religion}"
    
    # log judge decision
    decision = (f"Evaluator: Science scored {science_score}, "
                f"Religion scored {religion_score}. {highlight}")
    state["messages"].append(decision)
    
    # advance round or end debate
    if state["round"] >= state["max_rounds"]:
        state["next"] = END
    else:
        state["round"] += 1
        state["next"] = "science"
    
    return state

In [ ]:

workflow = StateGraph()
workflow.add_node("science", science_agent)
workflow.add_node("religion", religion_agent)
workflow.add_node("evaluator", evaluator_agent)
workflow.set_entry_point("science")

topic = input("Enter the debate topic (e.g., Education vs Money): ")

sides = {
    "science": topic.split(" vs ")[0] if "vs" in topic else "Education",
    "religion": topic.split(" vs ")[1] if "vs" in topic else "Money"
}

state = DebateState(
    messages=[],
    next=workflow.entry_point,
    round=1,
    max_rounds=5,
    topic=topic,
    sides=sides,
    scores={"science": 0, "religion": 0},
    highlights={"science": "", "religion": ""}
)

while state["next"] != END:
    agent_func = workflow.get_node(state["next"])
    state = agent_func(state)
    print(state["messages"][-1])   

print("\n--- FINAL RESULTS ---")
print(f"Science total: {state['scores']['science']} points")
print(f"Religion total: {state['scores']['religion']} points")

if state["scores"]["science"] > state["scores"]["religion"]:
    print("WINNER: Science side")
elif state["scores"]["science"] < state["scores"]["religion"]:
    print("WINNER: Religion side")
else:
    print("DRAW!")

print("\n--- BEST HIGHLIGHTS ---")
print(f"Science: {state['highlights']['science']}")
print(f"Religion: {state['highlights']['religion']}")


# 3. workflow assistant
input: user describes a task (e.g., “plan a study routine”).
graph:

- node 1 extracts subtasks

- node 2 searches web (or dummy data)

- node 3 synthesizes final plan

# 4. self-healing coder
a mini system where you feed code to one node, another node checks errors, and a third node proposes fixes. graph manages looping until code is “ok”.

# 5. daily journal summarizer
user dumps daily notes. one node organizes events, another node extracts key emotions, last node writes “insight of the day”. over time, graph keeps a memory chain.

# 6. task router
user says something, graph decides:

if it’s a factual q → send to retrieval node

if it’s casual chat → send to smalltalk node

if it’s a todo item → log into a simple json file.
perfect to practice conditional edges in langgraph.

# 7. multi-step reasoning tutor
user asks a math/logic q.
graph:

step 1 → rephrase problem

step 2 → propose reasoning steps

step 3 → verify with another node

step 4 → deliver clean final solution.